In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp template.formatter

In [ ]:
# export
# hide
from pathlib import Path
from typing import Dict, Union, List
from fastcore.script import call_parse, Param, store_true
import zipfile
import tempfile
import urllib
from string import Template
import re
import giturlparse
import subprocess

In [ ]:
# hide
# test imports
from pprint import pprint
import os

# Template Formatter

The template formatter generates a plugin from a template that sets up setup, docker and CI configurations for you.

You can list the available templates with:

```
plugin_from_template --list
```

Example usage:
```
plugin_from_template --template classifier_plugin --description "My Classifier Plugin"
```

In [ ]:
# export
# hide
TEMPLATE_URL = "https://gitlab.memri.io/memri/plugin-templates/-/archive/dev/plugin-templates-dev.zip"
TEMPLATE_BASE_PATH = "plugin-templates-dev"

## Utility functions -

In [ ]:
# export
# hide

# If the owner of the repository is one of these groups, the CLI requires an additional `user` argument
GITLAB_GROUPS = ["memri", "plugins"]

def get_remote_url(path="."):
    path = Path(path)
    url = subprocess.getoutput([f"cd {path} && git config --get remote.origin.url"])
    if not url:
        raise ValueError(f"'{path}' is not an initialized git repository")
    parsed = giturlparse.parse(url)
    repo_url = parsed.url2https
    if repo_url.endswith(".git"):
        repo_url = repo_url[:-4]
    return repo_url

def infer_git_info(url):
    parsed = giturlparse.parse(url)
    return parsed.owner, parsed.repo

In [ ]:
# hide
remote_url = get_remote_url()
repo_owner, repo_name = infer_git_info(remote_url)

assert repo_owner == "memri"
assert repo_name == "pymemri"

In [ ]:
# export
# hide
def str_to_identifier(s, lower=True):
    result = re.sub("\W|^(?=\d)", "_", s)
    if lower:
        result = result.lower()
    return result

def reponame_to_displayname(reponame: str) -> str:
    return re.sub("[-_]+", " ", reponame).title()

def download_plugin_template(
    template_name: str, url: str = TEMPLATE_URL, base_path: str = TEMPLATE_BASE_PATH
):
    base_path = str(Path(base_path) / template_name)
    zip_path, _ = urllib.request.urlretrieve(url)
    with zipfile.ZipFile(zip_path, "r") as f:
        result = {name: f.read(name) for name in f.namelist() if base_path in name}
    if len(result) == 0:
        raise ValueError(f"Could not find template: {template_name}")
    result = {k.replace(base_path, "").strip("/"): v.decode("utf-8") for k, v in result.items() if v}
    return result


def get_templates(url: str = TEMPLATE_URL) -> List[str]:
    zip_path, _ = urllib.request.urlretrieve(url)
    with zipfile.ZipFile(zip_path, "r") as f:
        files_split = [name.split("/") for name in f.namelist()]
        result = [fn[1] for fn in files_split if fn[-1] == '' and len(fn) == 3]
    return result

In [ ]:
# hide
assert len(get_templates())

In [ ]:
# hide
assert str_to_identifier("My Plugin") == "my_plugin"

template = download_plugin_template("classifier_plugin")
assert len(template)
pprint(list(template.keys()))

['$package_name/model.py',
 '$package_name/plugin.py',
 '$package_name/schema.py',
 '$package_name/utils.py',
 '.gitignore',
 '.gitlab-ci.yml',
 'Dockerfile',
 'LICENSE.txt',
 'README.md',
 'metadata.json',
 'setup.cfg',
 'setup.py',
 'tests/test_plugin.py',
 'tools/preload.py']


## Template Formatter

In [ ]:
# export
class TemplateFormatter:
    def __init__(
        self,
        template_dict: Dict[str, str],
        replace_dict: Dict[str, str],
        tgt_path: Union[str, Path],
        verbose: bool = False,
    ):
        self.template_dict = template_dict
        self.tgt_path = Path(tgt_path)
        self.replace_dict = replace_dict
        self.verbose = verbose

    def format_content(self, content):
        return Template(content).safe_substitute(self.replace_dict)

    def format_path(self, path):
        new_path = Template(path).safe_substitute(self.replace_dict)
        return self.tgt_path / new_path

    def format_file(self, filename, content):
        new_path = self.format_path(filename)
        new_content = self.format_content(content)
        new_path.parent.mkdir(exist_ok=True, parents=True)
        if self.verbose:
            print(f"Formatting {filename} -> {new_path}")
        with open(new_path, "w", encoding="utf-8") as f:
            f.write(new_content)

    def format(self):
        for filename, content in self.template_dict.items():
            self.format_file(filename, content)

## Template CLI

With the `plugin_from_template` CLI, you can easily create a plugin where all CI pipelines, docker files, and test setups are configured for you. Multiple templates are available, to see the complete list use:

`plugin_from_template --list_templates`

In [ ]:
# export
# hide
def get_template_replace_dict(
    repo_url=None, user=None, plugin_name=None, package_name=None, description=None
):
    if repo_url is None:
        repo_url = get_remote_url()

    try:
        repo_owner, repo_name = infer_git_info(repo_url)
    except ValueError:
        url_inf, owner_inf, name_inf = None, None, None
        print("Could not infer git information from current directory, no initialized repository found.")

    if repo_url is None:
        repo_url = url_inf

    if user is None:
        if repo_owner in GITLAB_GROUPS:
            user = None
        else:
            user = repo_owner

    if plugin_name is None:
        if repo_name is None:
            print("henk")
            plugin_name = None
        else:
            plugin_name = reponame_to_displayname(repo_name)

    if package_name is None:
        if repo_name is None:
            package_name = None
        else:
            package_name = str_to_identifier(repo_name)

    return {
        "user": user,
        "package_name": package_name,
        "plugin_name": plugin_name,
        "repo_name": repo_name,
        "repo_url": repo_url,
        "description": str(description),
    }

In [ ]:
# export
@call_parse
def plugin_from_template(
    list_templates: Param("List available plugin templates", store_true) = False,
    user: Param("Your Gitlab username", str) = None,
    repo_url: Param("The url of your empty Gitlab plugin repository", str) = None,
    plugin_name: Param("Display name of your plugin", str) = None,
    template_name: Param(
        "Name of the template, use `list_templates` to see all available options"
    ) = "basic",
    package_name: Param("Name of your plugin python package", str) = None,
    description: Param("Description of your plugin", str) = None,
    target_dir: Param("Directory to output the formatted template", str) = ".",
):
    if list_templates:
        print("Available templates:")
        for template in get_templates():
            print(template)
        return

    template = download_plugin_template(template_name)

    tgt_path = Path(target_dir)
    replace_dict = get_template_replace_dict(
        repo_url=repo_url,
        user=user,
        plugin_name=plugin_name,
        package_name=package_name,
        description=description,
    )

    formatter = TemplateFormatter(template, replace_dict, tgt_path)
    formatter.format()

    print(f"Created `{replace_dict['plugin_name']}` using the {template_name} template.")

In [ ]:
!plugin_from_template --list_templates

Available templates:
basic
classifier_plugin


In [ ]:
# hide
template = download_plugin_template("classifier_plugin")
replace_dict = {
    "user": "eelcovdw",
    "repo_name": "sentiment-plugin",
    "package_name": "sentiment_plugin",
    "plugin_name": "Sentiment Plugin",
    "description": "Predict sentiment on text messages"
}

with tempfile.TemporaryDirectory() as result_path:
    result_path = Path(result_path)
    formatter = TemplateFormatter(template, replace_dict, result_path)
    formatter.format()
    created_files = [f for f in result_path.rglob("*") if not os.path.isdir(f)]
    
    contents = {}
    for fn in created_files:
        with open(fn, "r") as f:
            contents[str(fn)] = f.read()

print("Created files:")  
pprint(created_files)
assert len(template) == len(created_files)

Created files:
[Path('/tmp/tmpzeq5mdrd/setup.py'),
 Path('/tmp/tmpzeq5mdrd/Dockerfile'),
 Path('/tmp/tmpzeq5mdrd/.gitignore'),
 Path('/tmp/tmpzeq5mdrd/README.md'),
 Path('/tmp/tmpzeq5mdrd/.gitlab-ci.yml'),
 Path('/tmp/tmpzeq5mdrd/metadata.json'),
 Path('/tmp/tmpzeq5mdrd/LICENSE.txt'),
 Path('/tmp/tmpzeq5mdrd/setup.cfg'),
 Path('/tmp/tmpzeq5mdrd/sentiment_plugin/model.py'),
 Path('/tmp/tmpzeq5mdrd/sentiment_plugin/schema.py'),
 Path('/tmp/tmpzeq5mdrd/sentiment_plugin/utils.py'),
 Path('/tmp/tmpzeq5mdrd/sentiment_plugin/plugin.py'),
 Path('/tmp/tmpzeq5mdrd/tools/preload.py'),
 Path('/tmp/tmpzeq5mdrd/tests/test_plugin.py')]


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted Untitled.ipynb.
Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted index.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.credentials.ipynb.
Converted plugin.authenticators.oauth.ipynb.
Converted plugin.listeners.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.states.ipynb.
Converted plugins.authenticators.password.ipynb.
Converted pod.api.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted template.config.ipynb.
Converted template.formatter.ipynb.
Converted test_schema.ipynb.
Converted test_utils.ipynb.
